In [ ]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.util import img_as_ubyte
from scipy.ndimage import gaussian_filter

# Notebook-friendly plot display
%matplotlib inline

In [ ]:
# Set your file path to singular .tif file
tif_path = 'example-data/example.tif'
# Use the nth frame for testing and select fluorescence channel
frame_index = 0
channel_index = 1

# Load TIFF without modifying it
with tifffile.TiffFile(tif_path) as tif:
    arr = tif.asarray()

# Handle multi-channel stack or single frame
if arr.ndim == 4:
    if arr.shape[1] <= 4:  # shape: (frames, channels, H, W)
        fl_image = arr[frame_index, channel_index]
    else:  # shape: (channels, frames, H, W)
        fl_image = arr[channel_index, frame_index]
elif arr.ndim == 3:
    fl_image = arr
else:
    raise ValueError("Unexpected TIFF shape:", arr.shape)

plt.imshow(fl_image, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Contrast stretch
p_low, p_high = np.percentile(fl_image, (1, 99))
contrast_stretched = exposure.rescale_intensity(fl_image, in_range=(p_low, p_high))

# Denoise (Gaussian blur)
denoised = gaussian_filter(contrast_stretched, sigma=1.0)

# Normalize for colormap
normalized = img_as_ubyte(denoised / np.max(denoised))

plt.imshow(normalized)
plt.axis('off')
plt.show()